
# **CLOUD SIDE**

In [ ]:
#!!!!Please note that this code is only for demonestrating the interactions between the cloud and clients, the process of sorting and managing patients order in the queue
#was simplified for the sake of simplicity!!!!!

''' 
Here we have a Virtual-queue for each hospital (in a region) 
for each patient depending on the severity of there case: they'll be added to the nearest ER VQ
Some basic ops will also be available like remove patient after being served etc
The queue shall always be ordered based on FCFS and higher severity has to be given higher prioirety 
'''
import numpy as np
Max_ERs= 3
#assign a unique code to each ER room! the third number is the average service time (time spent at the doctor's clinic)
# means that the first Queue in the list is for Carelton's ER

ERsLookup = {
  'NMC':0,
  'Medi':1, 
  'Queens':2}
#in minutes
AvgServiceTimeLookup = {
  0:20,
  1:30,
  2:10
  }
#assumption here is that only 3 ERs are available
class CloudWaitRooms:
    
    WaitRooms =[[]] * 3#list of lists , 3 ERs so 3 indexed from 0 to 2  
    def SelectBestER(self, patientid,patientTriagescore):
        ###Here, we get the closest ERs using "types.TYPE_HOSPITAL"
        ###for simplicity we'll assume that all 3 ERs are close to the patient
      
        #here's the list of all potential ERs, need to score them some how, then see which one offers minmum wait time and then return the best fit
        #CHANGE THIS CZ THE APPENDED ITEMS  WILL GO AFTER THE ZERO COLUMNS!:(
        WaiTimeScores= np.zeros((Max_ERs))#create an array of zeros
        #####for simplicity we'll assume that the ERs are sorted by distance 
        for queueindex in range(3):
           
           TempQ= list(self.WaitRooms[queueindex])
           TempQ.append((patientTriagescore,patientid))
           TempQ.sort()
           AvgServiceTime=AvgServiceTimeLookup.get(queueindex)
           index = TempQ.index((patientTriagescore,patientid))
           Waitime = AvgServiceTime * (index+1)
           WaiTimeScores = np.append(WaiTimeScores,Waitime)#each ER and its expected wait time 
           
        #return min in WaiTimeScores
        index_of_best_Q= np.argmin(WaiTimeScores)
        
        
        print(self.Addpatient(index_of_best_Q, patientid,patientTriagescore))
	      
        
    def Addpatient(self, queueindex, patientid, patientTriagescore):
        '''
           here we need ad a  petient with patientid as their ID number, queueindex as the best ER, patientTriagescore the score sent from the edge
        '''
        self.WaitRooms[queueindex].append((patientTriagescore,patientid))#add the patient to the corresponding ER (good here to make sure the patientid not duplicate)
        self.WaitRooms[queueindex].sort() #set the patient where they should be in queue, remember python executes stable sorting for duplicate values so the exsting values wont be affected !
        #will use this sorting function for the sake of simplicity 
        print("Your expected wait time is: " + str(self.ExpectedWaitTime(queueindex, patientid,patientTriagescore )))
        print("The selected ER is: ")
        print(list(ERsLookup.keys())[list(ERsLookup.values()).index(queueindex)])
        return "Patient added successfully"
        
    
    def ExpectedWaitTime(self,queueindex, patientid,triagescore ):
        #should run on a scheduled bases to keep patients updated
        #Tell the patient how long they'll have to wait before they can be seen by a doctor
        #get the average service time of the selected ER
        AvgServiceTime=AvgServiceTimeLookup.get(queueindex)
        index = self.WaitRooms[queueindex].index((triagescore,patientid))
        return AvgServiceTime * (index+1)
        

    def RemovePatient(self, queueindex, patientid,triagescore):
        self.WaitRooms[queueindex].remove((triagescore,patientid))
        #do we need to sort after that?
        return 'Patient removed from queue successfully'
    
        
    
        


# **EDGE SIDE**

In [ ]:
#DEFINE A CLOUD INSTANCE
CloudWaitRoomsobject = CloudWaitRooms()

TriageScoreLookup = {
  'Eczema':0,
  'SkinBurns1':1, 
  'SkinBurn3':5}

class Edge:
      #constructor!
      def __init__(self, ID):
          #When the Edge is being configured for the first time set the patient ID
          ###customise this based on the project scope: cover one patient or group of patients from the same household?
          self.patient_id = ID
      def Diagnose(self, image_from_sensor):
          Triage_Score= 5 #there has to be a defined scale , what numbers are reskier than others?
          #for the sake of simplicity we'll return the same score to all patients 
          ###how can this score be computed?
          #each illness is mapped to a number in the scale
          return Triage_Score
      def Send_request_to_cloud(self,Score):
          
          WaitTime = CloudWaitRoomsobject.SelectBestER(self.patient_id,Score)
          
          
       
    